In [2]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


In [2]:
from mip import Model, xsum, minimize, CBC, OptimizationStatus, BINARY
from itertools import product
import matplotlib.pyplot as plt
from math import sqrt
import numpy as np

#numero de clientes
n = 16
N = range(n)

#conjunto de veiculos
m = 3

#quantidade que os veículos suportam
q = 19

# posicao geografica dos nos
cx = np.array([25.00, 25.00, 27.00, 3.00, 17.00, 33.00, 32.00, 26.00, 20.00, 31.00, 23.00, 38.00, 14.00, 45.00, 9.00, 19.00])
cy = np.array([25.00, 46.00, 39.00, 10.00, 20.00, 7.00, 5.00, 22.00, 31.00, 36.00, 45.00, 23.00, 28.00, 21.00, 14.00, 42.00])

# demanda
d = np.array([ 0, 4, 5, 2, 3, 3, 6, 3, 3, 2, 3, 4, 1, 2, 2, 3])


# lista com os arcos
A = [(i,j) for (i,j) in product(N,N) if i != j]

# distancia Euclidiana entre os nos
c = [ [ sqrt( (cx[i] - cx[j])**2 + (cy[i] - cy[j])**2 ) for j in N ] for i in N]

#declaracao do modelo
model = Model('Problema do roteamento de veiculos capacitado',solver_name=CBC)

# x_ij igual a 1 se no j e visitado imediatamente apos no i; 0, caso contrario
x = {(i,j) : model.add_var(var_type=BINARY) for (i,j) in A}

# f_ij qtde de produto ficticio transportada pelo caixeiro viajante no arco (i,j)
f = {(i,j) : model.add_var(lb=0.0) for (i,j) in A}


model.objective = minimize(xsum(c[i][j] * x[i,j] for (i,j) in A))


for i in N:
    model += xsum(x[ii,j] for (ii,j) in A if i == ii and i != 0) == 1
for j in N:
    model += xsum(x[i,jj] for (i,jj) in A if j == jj and j != 0) == 1

model += xsum(x[ii,j] for (ii,j) in A if ii == 0) == m

model += xsum(x[i,jj] for (i,jj) in A if jj == 0) == m

model += xsum(f[i,jj] for (i,jj) in A if i == 0) == xsum(demanda for index, demanda in enumerate(d) if index != 0)


# restricao de balanco de fluxo: o caixeiro chega em um no com uma determinada
# quantidade de produto ficticio, deixa uma unidade deste produto, e sai
# deste no carregando uma unidade a menos.

for j in N:
  if j != 0:
    model += xsum(f[i,jj] for (i,jj) in A if j == jj) == d[j] + xsum(f[jj,i] for (jj,i) in A if j == jj)

# restricao de ativacao do arco: o caixeiro so pode usar o arco (i,j)
# para carregar produto ficticio se o arco for contabilizado na funcao objetivo
for (i,j) in A:
  model += f[i,j] <= q * x[i,j]

status = model.optimize()
print(status)

if status == OptimizationStatus.OPTIMAL:
  print("Custo da rota: {:12.2f}".format(sum([c[i][j] * x[i,j].x for (i,j) in A])))
  ini_rota = [j for (i,j) in A if i == 0 and x[i,j].x > 1e-6]
  for idx,r in enumerate(ini_rota):
      print("Rota {:2d}: {:2d} ".format(idx,0),end='')
      cur_no = r
      while True:
          print("{:3d} ".format(cur_no),end='')
          cur_no = [j for (i,j) in A if i == cur_no and x[i,j].x > 0.9][0]
          if cur_no == 0:
              break;
      print("{:2d} ".format(0))

OptimizationStatus.OPTIMAL
Custo da rota:       174.63
Rota  0:  0   7   6   5  13  11  0 
Rota  1:  0   8  12   3  14   4  0 
Rota  2:  0  15  10   1   2   9  0 
